# 10. Hébergement Local de Modèles Génératifs

**Durée estimée** : 60 minutes

**Prérequis** : Notebook 1 (OpenAI Intro), Docker, GPU (recommandé)

---

## Objectifs

Ce notebook explore l'hébergement **local** de LLMs via des serveurs compatibles OpenAI API :

1. **Configuration multi-endpoints** : Gérer plusieurs modèles/serveurs
2. **vLLM et Ollama** : Serveurs d'inférence populaires
3. **DeepSeek R1** : Modèle raisonnant local (alternative à o1)
4. **Qwen 2.5** : Tool calling et multimodal local
5. **Benchmarking** : Comparaison performances et coûts

---

## Pourquoi héberger localement ?

| Aspect | Cloud (OpenAI) | Local (vLLM/Ollama) |
|--------|----------------|---------------------|
| **Coût** | Par token ($) | Fixe (matériel + électricité) |
| **Latence** | Réseau + queue | Direct GPU |
| **Confidentialité** | Données envoyées | Données locales |
| **Disponibilité** | Dépend du service | 100% contrôle |
| **Modèles** | Limité au catalogue | Open-source illimité |

---

## Modèles locaux recommandés (2025-2026)

| Modèle | Taille | VRAM | Capacités |
|--------|--------|------|-----------|
| **DeepSeek R1** (distill) | 8B-70B | 8-48GB | Raisonnement, code |
| **Qwen 2.5** | 7B-72B | 8-48GB | Tool calling, multimodal |
| **Llama 3.1** | 8B-70B | 8-48GB | Généraliste |
| **Mistral/Mixtral** | 7B-8x7B | 8-48GB | Code, MoE |

---

## Installation & Import

On installe/importe ce qui est nécessaire :
- `requests` pour les appels HTTP bruts,
- `openai` version 1.0.0+,
- `semantic-kernel` si on veut tester SK,
- d'autres libs selon besoin (json, time, etc.).

## Concepts Clés

### vLLM (Very Large Language Models)

**vLLM** est un serveur d'inférence haute performance pour les LLMs :

- **PagedAttention** : Gestion optimisée de la mémoire GPU (KV cache)
- **Batching continu** : Traite plusieurs requêtes simultanément
- **Compatible OpenAI API** : Drop-in replacement pour les applications existantes
- **Support multi-GPU** : Tensor parallelism et pipeline parallelism

**Cas d'usage idéaux** :
- Production avec fort trafic (>100 req/min)
- Applications nécessitant faible latence
- Déploiement multi-GPU pour grands modèles (>70B paramètres)

### Ollama

**Ollama** est une plateforme de déploiement simplifiée pour LLMs locaux :

- **Installation ultra-simple** : Une commande pour démarrer
- **Gestion de modèles** : Téléchargement et versioning automatiques
- **Quantization** : Support Q4, Q5, Q8 pour réduire l'empreinte mémoire
- **API REST** : Compatible OpenAI + API native Ollama

**Cas d'usage idéaux** :
- Prototypage rapide et développement local
- Machines avec GPU limité (8-16GB VRAM)
- Applications mono-utilisateur ou faible trafic

### Comparaison vLLM vs Ollama

| Aspect | vLLM | Ollama |
|--------|------|--------|
| **Performance** | Excellent (batching optimisé) | Bon (single request) |
| **Setup** | Complexe (Docker + config) | Simple (1 commande) |
| **VRAM requis** | 16-24GB+ | 8GB+ (avec quantization) |
| **Multi-GPU** | Support natif | Limité |
| **Gestion modèles** | Manuel (HuggingFace) | Automatique (registry) |

### Endpoints OpenAI-compatibles

Les deux serveurs exposent les mêmes endpoints que l'API OpenAI :

-  : Conversations chat
-  : Complétion de texte brut
-  : Liste des modèles disponibles
-  : Génération d'embeddings (vLLM uniquement)

**Avantage clé** : Code portable entre cloud (OpenAI) et local (vLLM/Ollama)




In [ ]:
%pip install requests aiohttp openai tiktoken semantic-kernel python-dotenv anyio httpx httpcore


import os
import requests
import time
import json
import getpass
import openai


print("Importations OK.")

## Journalisation colorée

Nous allons utiliser un logger (via le module `logging`) configuré avec un
**ColorFormatter** pour afficher les messages en couleur dans la console ou la
sortie de Jupyter :

- Les **informations** et étapes réussies apparaîtront en vert (niveau `INFO`).
- Les **erreurs** seront en rouge (niveau `ERROR`).
- Les avertissements (`WARNING`) ou messages de debug (`DEBUG`) auront également leurs couleurs.


In [ ]:
import logging
class ColorFormatter(logging.Formatter):
    """
    Un formatter coloré pour rendre les logs plus lisibles.
    """
    colors = {
        'DEBUG': '\033[94m',
        'INFO': '\033[92m',
        'WARNING': '\033[93m',
        'ERROR': '\033[91m',
        'CRITICAL': '\033[91m\033[1m'
    }
    reset = '\033[0m'

    def format(self, record: logging.LogRecord) -> str:
        msg = super().format(record)
        return f"{self.colors.get(record.levelname, '')}{msg}{self.reset}"

logger = logging.getLogger("Local Llama")
logger.setLevel(logging.DEBUG)  # Peut être paramétré via .env ou variable

if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setLevel(logging.DEBUG)
    formatter = ColorFormatter(
        fmt="%(asctime)s [%(levelname)s] %(name)s - %(message)s",
        datefmt="%H:%M:%S"
    )
    handler.setFormatter(formatter)
    logger.addHandler(handler)

logger.info("Configuration initiale terminée.")

## Configuration et définition dynamique des endpoints

Pour simplifier la configuration de nos endpoints (URL d’API, clés d’API, modèles, etc.), nous allons externaliser ces informations dans un fichier `.env` placé à la racine de notre projet ou dans un dossier sécurisé. Copiez le fichier .env.example et renommez le fichier résultant en .env avant de le personnaliser.

### Déclaration dans `.env`

On déclare, par exemple, un premier endpoint dans des variables d’environnement :

```
OPENAI_ENDPOINT_NAME=OpenAI
OPENAI_BASE_URL=https://api.openai.com/v1
OPENAI_API_KEY=sk-abcd1234ABCD1423
OPENAI_CHAT_MODEL_ID=gpt-5-mini
```

Et si l’on souhaite tester plusieurs endpoints (ex. mini, medium, large), on ajoute un suffixe `_2`, `_3`... :

```
OPENAI_ENDPOINT_NAME_2=local-mini
OPENAI_BASE_URL_2=https://api.mini.yourdomain.com/v1
OPENAI_API_KEY_2=sk-MINI-SECRET-KEY
OPENAI_CHAT_MODEL_ID_2=unsloth/DeepSeek-R1-Distill-Llama-8B-bnb-4bit

OPENAI_ENDPOINT_NAME_3=medium
OPENAI_BASE_URL_3=https://api.medium.text-generation-webui.myia.io/v1
OPENAI_API_KEY_3=sk-MEDIUM-SECRET-KEY
OPENAI_CHAT_MODEL_ID_3=unsloth/DeepSeek-R1-Distill-Qwen-14B-bnb-4bit
```

Ainsi, chacun de ces blocs définit un **endpoint** : un service local ou distant OpenAI-compatible (ex. Oobabooga ou vLLM).  

### Lecture et création automatique dans le notebook

Dans le notebook, nous allons **lire** ces variables pour construire la liste `endpoints`. Chacun contient :

- `name` : un label descriptif (ex. `micro`, `mini`, etc.),
- `api_base` : l’URL de base de l’API (ex. `https://api.micro.text-generation-webui.myia.io/v1`),
- `api_key` : la clé API (fournie par votre conteneur ou config),
- `model` (optionnel) : si le modèle n’est pas fourni, nous pourrons interroger `/models` pour récupérer le nom du (ou des) modèle(s) disponibles.

Grâce à cette configuration dynamique, on peut aisément **alterner** entre différents backends (p. ex. Oobabooga ou vLLM) ou **interroger plusieurs endpoints** pour **comparer leurs performances**.

In [ ]:
from dotenv import load_dotenv
load_dotenv('../.env')  # Charge les variables du fichier .env (dans le répertoire parent GenAI/)


import os

def get_optional_env(var_name, default_value=None):
    """
    Récupère la valeur de la variable d'env `var_name`, 
    ou la valeur par défaut `default_value` si non définie.
    """
    val = os.getenv(var_name)
    if val is None or val.strip() == "":
        return default_value
    return val.strip()

def load_endpoint(index=1):
    """
    Lit un ensemble de variables d'environnement.
    - index=1 => variables : OPENAI_API_KEY, OPENAI_BASE_URL, etc.
    - index>1 => on suffixe : OPENAI_API_KEY_{index}, etc.
    Retourne un dict {name, api_base, api_key, model} ou None si 'api_key' manquant.
    """
    suffix = "" if index == 1 else f"_{index}"

    # Lecture des variables
    api_key = os.getenv(f"OPENAI_API_KEY{suffix}")
    if not api_key:
        return None  # pas de clé => on arrête

    name = get_optional_env(f"OPENAI_ENDPOINT_NAME{suffix}", default_value=f"openai{suffix}")
    base_url = get_optional_env(f"OPENAI_BASE_URL{suffix}", default_value="https://api.openai.com/v1")
    model_id = get_optional_env(f"OPENAI_CHAT_MODEL_ID{suffix}", default_value=None)

    return {
        "name": name,
        "api_base": base_url,
        "api_key": api_key,
        "model": model_id  # On pourra le compléter si None
    }


endpoints = []

# On tente successivement index=1,2,3... jusqu'à ce qu'on ne trouve plus OPENAI_API_KEY_{i}
for i in range(1, 10):  # max 9 endpoints, ajustez si besoin
    ep = load_endpoint(i)
    if ep is None:
        break
    endpoints.append(ep)

# Vérification (simple)
logger.info("=== Endpoints chargés ===")
for e in endpoints:
    logger.info(f"- name={e['name']}, base={e['api_base']}, key=(len={len(e['api_key'])}), model={e['model']}")

## Inspection des modèles disponibles

Nous allons appeler l'endpoint `/models` de chaque service 
pour récupérer la liste des modèles chargés côté serveur.

Si vous avez mis `model=None` dans la config, vous pourrez automatiquement 
récupérer le `model` à partir des données renvoyées. 


In [ ]:
import json
import time
import requests

def shrink_json(obj, skip_keys=None, max_str=500, _level=0, max_level=4):
    if skip_keys is None:
        skip_keys = set()

    if _level >= max_level:
        return "... (nested)"

    if isinstance(obj, dict):
        new_dict = {}
        for k, v in obj.items():
            if k in skip_keys:
                new_dict[k] = f"(skipped large data for key: {k})"
            else:
                new_dict[k] = shrink_json(v, skip_keys, max_str, _level+1, max_level)
        return new_dict

    elif isinstance(obj, list):
        return [
            shrink_json(x, skip_keys, max_str, _level+1, max_level)
            for x in obj
        ]

    elif isinstance(obj, str):
        if len(obj) > max_str:
            return obj[:max_str] + "... (truncated)"
        else:
            return obj
    else:
        return obj

def list_models(api_base, api_key):
    url = f"{api_base}/models"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    try:
        resp = requests.get(url, headers=headers, timeout=20)
        if resp.status_code == 200:
            return resp.json()  # dict
        else:
            return {"error": f"status={resp.status_code}", "text": resp.text}
    except Exception as e:
        return {"error": str(e)}

def update_endpoints_with_model():
    for ep in endpoints:
        logger.info(f"=== {ep['name']} : /models ===")
        start_time = time.time()

        info = list_models(ep["api_base"], ep["api_key"])
        elapsed_time = time.time() - start_time

        # On "rétrécit" le JSON pour éviter d'afficher les champs volumineux
        truncated_info = shrink_json(
            info,
            skip_keys={"profile_image_url", "raw_modelfile_content"},
            max_str=1000  # Tronque les chaînes > 1000 chars
        )

        # On journalise la version tronquée ou partiellement ignorée en DEBUG
        logger.debug("Réponse brute (tronquée): %s",
                     json.dumps(truncated_info, indent=2, ensure_ascii=False))

        if "error" in info:
            # En cas d'erreur, on logue au niveau ERROR
            logger.error(
                f"Échec de récupération /models (endpoint={ep['name']}): "
                f"{info['error']}, texte={info.get('text', '')}"
            )
        else:
            # Succès : on peut afficher le nombre de modèles
            data_models = info.get("data", [])
            logger.info(
                f"Réussite: {len(data_models)} modèle(s) listé(s) "
                f"(endpoint={ep['name']})"
            )

        logger.info(f"  -> Temps de réponse: {elapsed_time:.2f} secondes")

        # On met à jour ep["model"] si besoin
        if "error" not in info and ("model" not in ep or not ep["model"]):
            data_list = info.get("data", [])
            if data_list:
                first_model_id = data_list[0].get("id")
                ep["model"] = first_model_id
                logger.info(f"  -> ep['model'] défini à: {first_model_id}")
            else:
                logger.warning(
                    "  -> Aucune entrée 'data' dans la réponse pour définir ep['model']"
                )

# Test
update_endpoints_with_model()


### Analyse de la réponse /models

L'appel à l'endpoint `/models` permet d'**inspecter les modèles disponibles** sur chaque serveur :

**Informations récupérées** :
1. **Liste des modèles** : Identifiants (ex: `unsloth/DeepSeek-R1-Distill-Llama-8B-bnb-4bit`)
2. **Métadonnées** : Type de modèle, capacités, versions
3. **Statut** : Modèle chargé et prêt à recevoir des requêtes

**Observations clés** :
- **Temps de réponse** : Indicateur de la latence réseau et de la charge du serveur
- **Nombre de modèles** : Certains serveurs hébergent plusieurs modèles simultanément
- **Erreurs éventuelles** : Timeout, authentification échouée, serveur indisponible

**Utilité pratique** :
- **Auto-configuration** : Si `model=None` dans le config, on peut récupérer automatiquement le premier modèle disponible
- **Monitoring** : Vérifier que le serveur répond correctement
- **Découverte** : Lister les modèles accessibles sans connaître leurs noms à l'avance

**Champs potentiellement volumineux ignorés** :
- `profile_image_url` (images base64)
- `raw_modelfile_content` (configurations très longues)

Cela évite de saturer les logs tout en conservant les informations essentielles.

## Test brut via `requests.post`

Ce test vérifie le bon fonctionnement de l'endpoint OpenAI-compatible 
sans passer par la librairie `openai`. 

On envoie une requête minimaliste en JSON, 
puis on affiche la réponse brute.


In [ ]:
def test_brut_endpoints():
    """Test brut via requests.post() sur tous les endpoints."""
    for ep in endpoints:
        logger.info(f"\n=== Test HTTP brut pour {ep['name']} ===")
        
        url = f"{ep['api_base']}/chat/completions"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {ep['api_key']}"
        }
        payload = {
            "model": ep["model"],
            "messages": [
                {"role": "user", "content": "Bonjour, qui es-tu ?"}
            ],
            "max_completion_tokens": 200
        }
        
        logger.debug("  -> Envoi de la requête POST...")
        start_time = time.time()
        try:
            resp = requests.post(url, headers=headers, json=payload, timeout=30)
            elapsed_time = time.time() - start_time
            logger.debug(f"  -> Statut HTTP: {resp.status_code} (durée={elapsed_time:.2f}s)")
            
            # On essaie d'obtenir le JSON de la réponse
            try:
                resp_json = resp.json()
            except json.JSONDecodeError:
                logger.error(f"Réponse non-JSON:\n{resp.text[:200]}")
                continue
            
            # Affichage d’un extrait du JSON (en DEBUG, car potentiellement verbeux)
            dumped_json = json.dumps(resp_json, indent=2)
            logger.debug(f"Réponse (début): {dumped_json}")

            # Nombre de tokens si success
            tokens_used = None
            if resp.status_code == 200:
                if "usage" in resp_json:
                    tokens_used = resp_json["usage"].get("total_tokens")
                logger.info(
                    f"[OK] Réponse HTTP 200 en {elapsed_time:.2f}s, "
                    f"Tokens utilisés={tokens_used if tokens_used else 'N/A'}"
                )
            else:
                # On log au niveau ERROR pour signifier un souci
                logger.error(
                    f"[ERREUR] HTTP {resp.status_code}, texte={resp_json.get('message', resp.text)}"
                )
        except requests.exceptions.Timeout:
            logger.error("Timeout après 30s.")
        except Exception as e:
            logger.exception(f"Exception lors de la requête: {str(e)}")

# On exécute le test brut
test_brut_endpoints()


## Tokens et logits

### Tests de Tokenizers

Mise en place des fonctions utilitaires de **tokenization** (via `tiktoken` ou l'API `/tokenize` de vLLM), ainsi que quelques helpers pour **dé/retokenizer** en local (via l'API `/detokenize`) et pour **debugger** la segmentation et la correspondance de chaque token.


In [ ]:
import re
import requests
import tiktoken
import time
import logging


# ========================
# Helpers pour Tokenization
# ========================
def tokenize_sentence(api_base, api_key, sentence, model_fallback="gpt-5-mini"):
    """
    Tokenise 'sentence' en utilisant l'API /tokenize d'un backend vLLM ou, si l'API
    semble être celle d'OpenAI/Azure, utilise la librairie tiktoken.
    """
    official_providers = ("openai.com", "azure.com")
    if any(provider in api_base for provider in official_providers):
        logger.info("Endpoint='%s' => API OpenAI OFFICIELLE => usage local de tiktoken.", api_base)
        try:
            enc = tiktoken.encoding_for_model(model_fallback)
        except Exception:
            logger.warning("Modèle tiktoken inconnu (%s), fallback sur 'cl100k_base'.", model_fallback)
            enc = tiktoken.get_encoding("cl100k_base")
        token_ids = enc.encode(sentence)
        logger.info(f"Nombre de tokens (tiktoken): {len(token_ids)}")
        return token_ids

    logger.info("Endpoint='%s' => vLLM local => on utilise POST /tokenize", api_base)
    api_base_sanitized = re.sub(r"/v1/?$", "", api_base.rstrip("/"))
    url = f"{api_base_sanitized}/tokenize"
    headers = {
        "Authorization": f"Bearer {api_key}" if api_key else "",
        "Content-Type": "application/json"
    }
    payload = {"prompt": sentence}
    try:
        resp = requests.post(url, headers=headers, json=payload, timeout=10)
        if resp.status_code == 200:
            data = resp.json()
            tokens = data.get("tokens", [])
            count = data.get("count", 0)
            logger.info(f"Nombre de tokens (vLLM): {count}")
            return tokens
        else:
            logger.error(f"Erreur Tokenizer API: {resp.status_code}, {resp.text}")
            return None
    except Exception as e:
        logger.error(f"Exception lors de l'appel à /tokenize: {str(e)}")
        return None

def tokenize_vllm(api_base, api_key, text):
    """
    Appelle POST /tokenize sur un endpoint vLLM local.
    Retourne la liste d'IDs ou None en cas d'erreur.
    """
    base = re.sub(r"/v1/?$", "", api_base.rstrip("/"))
    url = f"{base}/tokenize"
    headers = {
        "Authorization": f"Bearer {api_key}" if api_key else "",
        "Content-Type": "application/json"
    }
    payload = {"prompt": text}
    resp = requests.post(url, headers=headers, json=payload, timeout=10)
    if resp.status_code == 200:
        data = resp.json()
        return data.get("tokens", None)
    else:
        print(f"[tokenize_vllm] Erreur {resp.status_code} => {resp.text}")
        return None

def detokenize_vllm(api_base, api_key, token_ids):
    """
    Appelle POST /detokenize sur un endpoint vLLM local.
    Retourne la chaîne correspondant à la liste de tokens.
    """
    base = re.sub(r"/v1/?$", "", api_base.rstrip("/"))
    url = f"{base}/detokenize"
    headers = {
        "Authorization": f"Bearer {api_key}" if api_key else "",
        "Content-Type": "application/json"
    }
    payload = {"tokens": token_ids}
    resp = requests.post(url, headers=headers, json=payload, timeout=10)
    if resp.status_code == 200:
        data = resp.json()
        return data.get("prompt", "")
    else:
        print(f"[detokenize_vllm] Erreur {resp.status_code} => {resp.text}")
        return None

def debug_token_mapping(api_base, api_key, text):
    """
    Tokenise 'text' et affiche pour chaque token son fragment via /detokenize.
    """
    tokens = tokenize_vllm(api_base, api_key, text)
    if not tokens:
        print("Pas de tokens ou échec.")
        return
    print(f"Liste des token_ids: {tokens}")
    print(f"Nombre de tokens = {len(tokens)}")
    for idx, tid in enumerate(tokens):
        sub = detokenize_vllm(api_base, api_key, [tid])
        print(f"[{idx}] token_id={tid} => {repr(sub)}")

def get_token_id_for_word(api_base, api_key, word, model_fallback="gpt-5-mini"):
    """
    Retourne le premier token ID obtenu en tokenisant 'word' sur l'endpoint donné.
    """
    tokens = tokenize_sentence(api_base, api_key, word, model_fallback)
    if tokens and len(tokens) > 0:
        return tokens[0]
    else:
        logger.warning(f"Aucun token obtenu pour le mot '{word}' sur l'endpoint '{api_base}'.")
        return None


# ============= Exemples d'utilisation =============
# 1) Test rapide de la "tokenize_sentence" sur tous endpoints
sampleSentence = "Bonjour ! Je suis un assistant virtuel, conçu pour répondre à vos questions et vous aider avec diverses informations. Comment puis-je vous aider aujourd'hui ?"
for ep in endpoints:
    logger.info(f"=== Test Tokenizer API (ou local tiktoken) pour endpoint '{ep['name']}' ===")
    tokens = tokenize_sentence(ep["api_base"], ep["api_key"], sampleSentence)
    if tokens:
        logger.info(f"Tokens générés: {tokens}")
    else:
        logger.warning(f"Échec de la tokenisation pour l'endpoint '{ep['name']}'.")

# 2) Test plus avancé: debug_token_mapping
sample_text = "</think>Wait,Alternatively,Hmm,But.\nBut wait,  But let me think again. Wait, Alternatively, Hmm, "
print(f"\n=== sample text: '{sample_text}' ===")
for ep in endpoints:
    print(f"\n=== Test sur endpoint '{ep['name']}' ===")
    debug_token_mapping(ep["api_base"], ep["api_key"], sample_text)



def get_token_ids_for_variants(api_base, api_key, word, model_fallback="gpt-5-mini"):
    """
    Pour un mot donné, retourne un ensemble des token IDs correspondant aux variantes
    possibles : sans espace, avec espace en préfixe, et avec majuscule/minuscule.
    Exemple pour "je": ["je", " Je", "Je", " je"].
    """
    variants = [word, " " + word, word.capitalize(), " " + word.capitalize()]
    token_ids = set()
    for variant in variants:
        tokens = tokenize_sentence(api_base, api_key, variant, model_fallback)
        if tokens and len(tokens) > 0:
            token_ids.add(tokens[0])
    if not token_ids:
        logger.warning(f"Aucun token obtenu pour les variantes du mot '{word}' sur l'endpoint '{api_base}'.")
    else:
        logger.info(f"Pour le mot '{word}', variantes {variants} -> token IDs: {token_ids}")
    return token_ids

# Exemple d'utilisation pour vérifier la tokenisation de "je"
sample_word = "je"
for ep in endpoints:
    logger.info(f"\n--- Endpoint: {ep['name']} ---")
    debug_token_mapping(ep["api_base"], ep["api_key"], sample_word)
    # Affiche les token IDs pour les variantes de "je"
    ids = get_token_ids_for_variants(ep["api_base"], ep["api_key"], sample_word)
    logger.info(f"Token IDs pour '{sample_word}' sur '{ep['name']}': {ids}")




### Interprétation du debug de tokenisation

La fonction `debug_token_mapping` affiche la correspondance exacte **token_id ↔ fragment de texte** :

**Exemple de sortie attendu** :
```
[0] token_id=1234 => '</think>'
[1] token_id=5678 => 'Wait'
[2] token_id=9012 => ','
[3] token_id=3456 => 'Alternatively'
...
```

**Enseignements** :

1. **Tokens spéciaux** : `</think>`, `Wait`, `Hmm` peuvent être des tokens uniques ou décomposés selon le modèle
2. **Espaces et ponctuation** : Notez comment les espaces sont traités (parfois fusionnés avec les mots, parfois séparés)
3. **Variabilité** : Le même texte peut produire des tokens différents selon l'endpoint (Llama vs Qwen vs GPT)

**Pourquoi c'est important ?**

- **Logit bias** : Pour biaiser un token, il faut connaître son ID exact
- **Optimization** : Certains modèles tokenisent plus efficacement (moins de tokens = moins de coût)
- **Debugging** : Comprendre pourquoi un prompt produit un nombre de tokens inattendu

**Cas d'usage** :
- Forcer ou interdire certains mots dans la génération (via `logit_bias`)
- Analyser les différences de tokenisation entre modèles
- Optimiser les prompts pour réduire la consommation de tokens

### Analyse des résultats logit_bias

Le test `logit_bias_consistency` évalue **3 aspects critiques** :

**1. Impact du logit_bias**
- Un `logit_bias` négatif (-100.0) sur les tokens de "je" devrait **fortement pénaliser** leur apparition
- La réponse AVEC bias devrait éviter d'utiliser "je" (ou ses variantes)
- Exemple attendu : "Bonjour ! En tant qu'assistant..." au lieu de "Bonjour, je suis..."

**2. Cohérence avec seed fixée**
- Deux appels identiques (`temperature=1.0`, `seed=42`) SANS bias devraient retourner la **même réponse**
- Si les réponses diffèrent, cela indique un problème de reproductibilité du serveur

**3. Validation du mécanisme**
- Si AVEC bias = SANS bias, le logit_bias n'a pas fonctionné (bug serveur ou modèle non compatible)
- Si AVEC bias ≠ SANS bias ET cohérence OK, le test est réussi

**Codes couleur des logs** :
- **Vert (INFO)** : Cohérence vérifiée, différence détectée → succès
- **Rouge (ERROR)** : Incohérence entre deux appels sans bias → problème
- **Orange (WARNING)** : Pas de différence détectée → logit_bias inefficace

**Implications** :
- Certains modèles locaux (Llama, Qwen) peuvent ne pas supporter `logit_bias`
- OpenAI officiel supporte bien ce paramètre
- Utile pour contrôler le vocabulaire (ex: interdire des mots sensibles)

### Test de requête avec des logit_bias

Nous présentons à présent un **exemple de test** pour le paramètre [`logit_bias`](https://platform.openai.com/docs/api-reference/chat/create#chat-create-logit_bias). Le principe : envoyer deux requêtes identiques vers chaque endpoint :

1. L’une comporte un champ `logit_bias` qui favorise ou pénalise un certain token ID.  
2. L’autre n’a pas de `logit_bias`.

On compare ensuite les réponses pour voir si l’application du biais a un effet tangible sur la génération. Dans la pratique, vous devrez adapter la valeur du token ID ciblé (`"13824"` ci-dessous) à la sortie que vous aurez obtenue dans la cellule précédente.


In [ ]:
def combine_message(msg):
    """
    Concatène 'reasoning_content' et 'content' d'un message pour obtenir
    une réponse complète.
    Si l'un des champs est None, il est remplacé par une chaîne vide.
    """
    if not msg:
        return ""
    reasoning = msg.get("reasoning_content") or ""
    content = msg.get("content") or ""
    return (reasoning + " " + content).strip()

def test_logit_bias_consistency():
    """
    Pour chaque endpoint, cette fonction effectue :
    
    1. Le calcul des token IDs pour le mot "je" en considérant les variantes
       ["je", " je", "Je", " Je"], en passant le nom du modèle pour le tokenizer.
    2. Un appel à l'API avec un logit_bias négatif (pour biaiser ces tokens).
    3. Deux appels identiques (même prompt, température, seed) sans logit_bias pour vérifier la cohérence.
    4. Si le message retourné contient "reasoning_content", celui-ci est concaténé avec "content".
    
    Les résultats sont loggués :
      - Si les deux appels sans logit_bias renvoient la même réponse, la cohérence est vérifiée.
      - Sinon, une erreur est logguée en rouge.
      - Une différence entre la réponse avec logit_bias et sans est également signalée.
    """
    logger.info("=== Test de logit_bias avec vérification de cohérence et concaténation ===")
    
    url_suffix = "/chat/completions"
    for ep in endpoints:
        logger.info(f"\n=== Test logit_bias sur endpoint '{ep['name']}' ===")
        url = f"{ep['api_base']}{url_suffix}"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {ep['api_key']}"
        }
        prompt = "Bonjour, qui es-tu?"
        
        # Calcul des token IDs pour les variantes de "je" en passant le modèle
        token_variants = ["je", " je", "Je", " Je", "Bonjour"]
        token_ids_set = set()
        model_name = ep.get("model", "gpt-5-mini")
        for variant in token_variants:
            tokens = tokenize_sentence(ep["api_base"], ep["api_key"], variant, model_fallback=model_name)
            if tokens:
                token_ids_set.update(tokens)
        if token_ids_set:
            logger.info(f"Pour le mot 'je', variantes {token_variants} -> token IDs: {token_ids_set}")
        else:
            logger.warning(f"Aucun token obtenu pour le mot 'je' sur l'endpoint '{ep['name']}'. Passage à l'endpoint suivant.")
            continue
        
        # Construction du logit_bias négatif sur ces tokens
        logit_bias = {str(tid): -100.0 for tid in token_ids_set}
        logger.info(f"Logit_bias appliqué: {logit_bias}")
        max_completion_tokens = 50
        temperature = 1.0
        
        payload_bias = {
            "model": ep["model"],
            "messages": [{"role": "user", "content": prompt}],
            "max_completion_tokens": max_completion_tokens,
            "temperature": temperature,
            "seed": 42,
            "logit_bias": logit_bias
        }
        payload_no_bias = {
            "model": ep["model"],
            "messages": [{"role": "user", "content": prompt}],
            "max_completion_tokens": max_completion_tokens,
            "temperature": temperature,
            "seed": 42
        }
        
        # Appel avec logit_bias
        logger.debug("Envoi de la requête AVEC logit_bias...")
        try:
            resp_bias = requests.post(url, headers=headers, json=payload_bias, timeout=30)
            logger.debug(f"Statut HTTP avec logit_bias: {resp_bias.status_code}")
            data_bias = resp_bias.json()
            message_bias = data_bias.get("choices", [{}])[0].get("message", {})
            result_bias = combine_message(message_bias)
        except Exception as e:
            result_bias = f"Erreur lors de la lecture de la réponse avec logit_bias: {e}"
        logger.info(f"Réponse avec logit_bias: {result_bias}")
        
        # Deux appels sans logit_bias pour vérifier la cohérence
        logger.debug("Envoi de la première requête SANS logit_bias...")
        try:
            resp_no_bias1 = requests.post(url, headers=headers, json=payload_no_bias, timeout=30)
            logger.debug(f"Statut HTTP sans logit_bias (1): {resp_no_bias1.status_code}")
            data_no_bias1 = resp_no_bias1.json()
            message_no_bias1 = data_no_bias1.get("choices", [{}])[0].get("message", {})
            result_no_bias1 = combine_message(message_no_bias1)
        except Exception as e:
            result_no_bias1 = f"Erreur lors de la première réponse sans logit_bias: {e}"
        logger.info(f"Réponse sans logit_bias (1): {result_no_bias1}")
        
        logger.debug("Envoi de la deuxième requête SANS logit_bias...")
        try:
            resp_no_bias2 = requests.post(url, headers=headers, json=payload_no_bias, timeout=30)
            logger.debug(f"Statut HTTP sans logit_bias (2): {resp_no_bias2.status_code}")
            data_no_bias2 = resp_no_bias2.json()
            message_no_bias2 = data_no_bias2.get("choices", [{}])[0].get("message", {})
            result_no_bias2 = combine_message(message_no_bias2)
        except Exception as e:
            result_no_bias2 = f"Erreur lors de la deuxième réponse sans logit_bias: {e}"
        logger.info(f"Réponse sans logit_bias (2): {result_no_bias2}")
        
        # Vérification de la cohérence
        if result_no_bias1 == result_no_bias2:
            logger.info("=> Cohérence vérifiée : les deux appels sans logit_bias avec la même seed renvoient la même réponse.")
        else:
            logger.error("=> ERREUR : Les deux appels sans logit_bias avec la même seed renvoient des réponses différentes!")
        
        # Comparaison entre la réponse avec logit_bias et sans
        if result_bias != result_no_bias1:
            logger.info("=> Différence détectée entre AVEC et SANS logit_bias.")
        else:
            logger.warning("=> Aucune différence détectée. Le logit_bias n'a pas eu d'effet apparent.")

# Exécute le test
test_logit_bias_consistency()


## Test avec la librairie `openai`

On reproduit un appel classique OpenAI, 
mais en changeant `openai.api_base` et `openai.api_key` 
pour chaque endpoint.


In [ ]:
from openai import OpenAI

def test_openai_chat(api_base, api_key, prompt, model):
    """
    Appel classique OpenAI, en utilisant la classe `OpenAI`
    et en gérant la journalisation via logger.
    """
    # Création du client OpenAI-compatible
    client = OpenAI(
        api_key=api_key,
        base_url=api_base
    )

    if not model:
        logger.error("[!] Modèle non défini.")
        raise ValueError("Modèle non défini")

    try:
        # Appel chat.completions
        logger.debug("Appel de client.chat.completions.create()...")
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_completion_tokens=500
        )
        content = response.choices[0].message.content
        tokens_used = response.usage.total_tokens if response.usage else None
        return content, tokens_used
    except Exception as e:
        logger.exception(f"Exception lors de l'appel OpenAI: {str(e)}")
        return None, None

def test_openai_endpoints():
    """Itère sur tous les endpoints et lance un prompt 'philosophie stoïcienne'."""
    for ep in endpoints:
        label_model = ep.get("model", "<aucun>")
        logger.info(f"\n=== Test openai pour {label_model} ({ep['name']}) ===")
        
        start_time = time.time()
        prompt = "Peux-tu résumer la philosophie stoïcienne en quelques lignes ?"
        content, tks = test_openai_chat(
            ep["api_base"],
            ep["api_key"],
            prompt,
            ep["model"]
        )
        elapsed_time = time.time() - start_time

        if content:
            logger.info(f" -> Réponse (début): {content[:200]}...")
            logger.info(f" -> Nb tokens: {tks}")
            logger.info(f" -> Temps écoulé: {elapsed_time:.2f} sec")
        else:
            logger.warning(" -> Pas de contenu (erreur ou exception).")

# On exécute le test
test_openai_endpoints()


## Test avec Semantic Kernel (optionnel)

Exemple d'intégration avec [Semantic Kernel](https://github.com/microsoft/semantic-kernel).
On crée un Kernel, on y ajoute un service chat OpenAI-like avec l'endpoint souhaité, 
et on exécute un prompt simple.


### Analyse du Function/Tool Calling

Le Function Calling permet au LLM de **déclencher des actions structurées** au lieu de générer du texte libre :

**Déroulement observé** :

1. **Déclaration de l'outil** : On définit `get_weather` avec son schéma JSON (paramètres location, unit)
2. **Prompt utilisateur** : "Donne-moi la météo pour Marseille en celsius"
3. **Réponse du modèle** : 
   - Le modèle détecte qu'il doit appeler `get_weather`
   - Il extrait automatiquement les arguments : `{"location": "Marseille", "unit": "celsius"}`
4. **Exécution côté client** : Notre fonction Python `get_weather()` est appelée avec ces arguments
5. **Résultat** : "Simulation: Météo à Marseille, unité=celsius, ciel dégagé."

**Avantages majeurs** :

- **Structuration** : Pas besoin de parser du texte libre (ex: "Il fait 20 degrés")
- **Fiabilité** : Les arguments sont validés par le schéma JSON
- **Automatisation** : Connexion directe à des APIs réelles (météo, calendrier, base de données)

**Endpoints compatibles** :
- OpenAI (natif)
- vLLM avec `--enable-auto-tool-choice` et `--tool-call-parser`
- Certains modèles locaux (Llama 3.1+, Qwen 2.5+)

**Warning possible** : Si un endpoint ne supporte pas le tool calling, il renverra du texte au lieu d'un `function_call`.

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    OpenAIChatCompletion,
    OpenAIChatPromptExecutionSettings
)
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.prompt_template.input_variable import InputVariable
from semantic_kernel.functions import KernelArguments
from openai import AsyncOpenAI
import asyncio

async def test_semantic_kernel():
    """
    Exécute un prompt via Semantic Kernel pour chaque endpoint,
    et journalise les résultats en couleur via `logger`.
    """
    for ep in endpoints:
        model_id = ep.get("model")
        api_key = ep["api_key"]

        logger.info(f"=== Test Semantic Kernel pour endpoint='{ep['name']}', model='{model_id}' ===")

        kernel = sk.Kernel()
        async_client = AsyncOpenAI(api_key=api_key, base_url=ep["api_base"])

        kernel.add_service(
            OpenAIChatCompletion(
                service_id="default",
                ai_model_id=model_id,
                async_client=async_client
            )
        )
        logger.debug("Service OpenAI ajouté au Kernel.")

        prompt_template = "Explique ce qu'est l'apprentissage profond (deep learning) en 500 mots."

        exec_settings = OpenAIChatPromptExecutionSettings(
            service_id="default",
            ai_model_id=model_id,
            max_completion_tokens=500,
        )

        pt_config = PromptTemplateConfig(
            template=prompt_template,
            name="deepLearningFunction",
            template_format="semantic-kernel",
            input_variables=[],
            execution_settings=exec_settings,
        )

        func = kernel.add_function(
            function_name="deepLearningFunction",
            plugin_name="defaultPlugin",
            prompt_template_config=pt_config
        )

        try:
            logger.info("  -> Exécution en cours (Semantic Kernel)...")
            start_time = time.time()
            result = await kernel.invoke(func, KernelArguments())
            elapsed = time.time() - start_time

            # Comptage approximatif de tokens
            tokens_count = len(str(result).split())
            speed = tokens_count / elapsed if elapsed > 0 else 0

            logger.info(f"  -> Résultat (début): {str(result)[:400]}...")
            logger.info(f"  -> Durée: {elapsed:.2f}s, Tokens={tokens_count}, speed={speed:.2f} tok/s")
        except Exception as e:
            logger.exception(f"  [!] Erreur SK sur endpoint='{ep['name']}': {str(e)}")


import nest_asyncio
nest_asyncio.apply()

await test_semantic_kernel()


## Test du Function/Tool Calling sur chaque endpoint

Avec vLLM, lorsqu’on démarre les containers avec `--enable-auto-tool-choice` et un `--tool-call-parser` adéquat,
le modèle peut déclencher automatiquement un « tool call » s’il juge qu’un outil est pertinent.  
On doit alors inclure un paramètre `tools` dans la requête, et indiquer `tool_choice="auto"` (ou un nom de fonction précis).

**Note** : Pour exécuter concrètement la fonction côté client Python, on doit définir une fonction Python qui correspond,
et réinjecter manuellement le résultat dans la conversation.  
Voici un exemple simplifié : on va appeler un `get_weather(location, unit)` sur *tous* les endpoints.


In [ ]:
import json
import openai
from openai import OpenAI

def get_weather(location: str, unit: str):
    """Exemple de fonction locale pour la météo."""
    return f"Simulation: Météo à {location}, unité={unit}, ciel dégagé."

def test_tool_calling():
    """
    Test du Function/Tool Calling pour chaque endpoint, en mode auto (tool_choice='auto').
    On journalise les étapes et le résultat.
    """
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Obtenir la météo pour un lieu donné",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string", "description": "Ex: 'Paris, France'"},
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
                    },
                    "required": ["location", "unit"]
                }
            }
        }
    ]

    user_message = "Bonjour, est-ce que tu peux me donner la météo pour Marseille en celsius ?"

    for ep in endpoints:
        logger.info(f"=== Test Tool Calling sur endpoint '{ep['name']}' ===")

        openai.api_base = ep["api_base"]
        openai.api_key  = ep["api_key"]

        client = OpenAI(
            api_key=ep["api_key"],
            base_url=ep["api_base"]
        )

        try:
            logger.debug("Appel chat.completions avec tool_choice='auto'")
            response = client.chat.completions.create(
                model=ep.get("model"),
                messages=[{"role": "user", "content": user_message}],
                tools=tools,
                tool_choice="auto",
            )
        except Exception as ex:
            logger.error(f"[!] Erreur lors de l'appel sur endpoint='{ep['name']}': {ex}")
            continue

        choice = response.choices[0]
        msg = choice.message

        text_content = msg.content if msg.content else ""
        tool_calls = msg.tool_calls

        logger.debug(f"Réponse textuelle (début): {text_content[:400]!r}")
        logger.debug(f"Contenu complet de la réponse: {json.dumps(response.to_dict(), indent=2)}")

        if tool_calls:
            logger.info(f"  -> {len(tool_calls)} tool_call(s) détecté(s) sur endpoint='{ep['name']}'.")
            for call in tool_calls:
                func_name = call.function.name
                args_str  = call.function.arguments or "{}"
                args_dict = json.loads(args_str)

                logger.info(f"     Fonction appelée: {func_name} | arguments={args_dict}")

                # Exécution locale simulée
                if func_name == "get_weather":
                    result = get_weather(**args_dict)
                    logger.info(f"     => Résultat simulé: {result}")
                else:
                    logger.warning(f"     => Fonction inconnue: {func_name}")
        else:
            logger.warning("  -> Aucun tool_call détecté dans la réponse.")

# On l’exécute
test_tool_calling()


### Interprétation du mode Reasoning

Le mode reasoning permet au modèle d'exposer son **raisonnement intermédiaire** :

**Question test** : "Combien font 253 * 73 - 287 ?"

**Réponse attendue** :

1. **Champ `reasoning_content`** (si supporté) :
   ```
   <think>
   Calculons d'abord 253 × 73 :
   253 × 70 = 17 710
   253 × 3 = 759
   Total : 17 710 + 759 = 18 469
   
   Ensuite, soustrayons 287 :
   18 469 - 287 = 18 182
   </think>
   ```

2. **Champ `content`** (réponse finale) :
   ```
   Le résultat est 18 182.
   ```

**Modèles compatibles** :
- **DeepSeek R1** : Natif avec balises `<think>...</think>`
- **o1/o3 (OpenAI)** : Reasoning intégré (mais non exposé publiquement dans l'API standard)
- **Qwen avec reasoning parser** : Nécessite `--enable-reasoning --reasoning-parser deepseek_r1`

**Avantages** :
- **Transparence** : On peut auditer le raisonnement
- **Détection d'erreurs** : Si le calcul intermédiaire est faux, on sait pourquoi
- **Pédagogie** : Utile pour l'enseignement (montrer les étapes)

**Limitation** :
- Consomme plus de tokens (le raisonnement compte dans le total)
- Pas supporté par tous les modèles/endpoints

## Test du Function/Tool Calling via semantic-kernel



### Analyse des résultats du benchmark

Le benchmark séquentiel évalue les **performances individuelles** de chaque endpoint :

**Métriques clés** :

1. **Temps moyen par requête** (`avg_time_s`) :
   - Indicateur de la latence du modèle
   - Dépend de : taille du modèle, quantization, charge serveur, bande passante

2. **Tokens par seconde** (`tokens_per_sec`) :
   - **Métrique de vitesse de génération**
   - Formule : `total_tokens / total_time`
   - Valeurs typiques :
     - Modèles 7B-8B (4-bit) : 20-50 tok/s
     - Modèles 14B (4-bit) : 10-25 tok/s
     - GPU puissant (RTX 4090) : 50-100+ tok/s

3. **Success rate** :
   - Nombre de requêtes réussies / total de requêtes
   - Un taux < 100% indique des timeouts ou erreurs

**Facteurs influençant les performances** :
- **Quantization** : 4-bit (bnb) vs 8-bit (fp8) vs 16-bit (fp16)
- **VRAM disponible** : Plus de VRAM = batch size plus grand
- **Context length** : Prompts longs = génération plus lente
- **KV cache** : Optimisation pour les conversations longues

**Comparaison typique** :
```
OpenAI (gpt-5-mini)    : ~15-20 tok/s (cloud, partagé)
Local mini (8B 4-bit)   : ~30-40 tok/s (dédié, RTX 3090)
Local medium (14B 4-bit): ~15-20 tok/s (dédié, RTX 3090)
```

**Conclusion** : Les modèles locaux peuvent être **plus rapides** que le cloud si bien configurés.

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    OpenAIChatCompletion,
    OpenAIChatPromptExecutionSettings
)
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.contents import ChatHistory
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.prompt_template.input_variable import InputVariable
from semantic_kernel.functions import KernelArguments
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.functions import KernelArguments, kernel_function
from openai import AsyncOpenAI
import asyncio
from typing import TYPE_CHECKING, Annotated


class MenuPlugin:
    """Plugin pour gérer un menu"""
    @kernel_function(description="Liste les specials")
    def get_specials(self) -> Annotated[str, "Describes specials"]:
        # print function call
        print("get_specials called")
        return "Special Soup: Clam Chowder\nSpecial Salad: Cobb Salad\nSpecial Drink: Chai Tea"
    @kernel_function(description="Donne le prix d'un item")
    def get_item_price(self, menu_item: Annotated[str, "nom de l'item"]) -> str:
         # print function call
        print("get_item_price called")
        return "$9.99"
    


async def test_semantic_kernel_plugin():
    """
    Exécute un prompt via Semantic Kernel pour chaque endpoint,
    et journalise les résultats en couleur via `logger`.
    """
    for ep in endpoints:
        model_id = ep.get("model")
        api_key = ep["api_key"]

        logger.info(f"=== Test Semantic Kernel pour endpoint='{ep['name']}', model='{model_id}' ===")

        kernel = sk.Kernel()
        kernel.add_plugin(MenuPlugin(), plugin_name="menu")
        async_client = AsyncOpenAI(api_key=api_key, base_url=ep["api_base"])

        kernel.add_service(
            OpenAIChatCompletion(
                service_id="default",
                ai_model_id=model_id,
                async_client=async_client
            )
        )
        
        settings2 = kernel.get_prompt_execution_settings_from_service_id(service_id="default")
        settings2.function_choice_behavior = FunctionChoiceBehavior.Auto()     
        
        logger.debug("Service OpenAI ajouté au Kernel.")

        AGENT2_NAME = "Host"
        AGENT2_INSTRUCTIONS = "Answer questions about the menu."
        agent2 = ChatCompletionAgent(
            kernel=kernel,
            name=AGENT2_NAME,
            instructions=AGENT2_INSTRUCTIONS,
            arguments=KernelArguments(settings=settings2),
        )
        chat_history = ChatHistory()
        user_msgs = [
            "Hello",
            "What is the special soup?",
            "What does it cost?",
            "Thanks",
        ]
        for user_input in user_msgs:
            chat_history.add_user_message(user_input)
            print(f"# User: '{user_input}'")
            agent_name = None
            try:
                logger.debug("Appel d'agent semantickernel avec plugin et tool_choice='auto'")
                async for content in agent2.invoke_stream(chat_history):
                    if not agent_name:
                        agent_name = content.name or AGENT2_NAME
                        print(f"# {agent_name}: '", end="")
                    if content.content.strip():
                        print(content.content, end="", flush=True)
            except Exception as ex:
                logger.error(f"[!] Erreur lors de l'appel sur endpoint='{ep['name']}': {ex}")
                continue
            
 
import nest_asyncio
nest_asyncio.apply()

await test_semantic_kernel_plugin()


## Test du mode « Reasoning Outputs »

Certains modèles (p. ex. DeepSeek R1) sont lancés avec `--enable-reasoning --reasoning-parser deepseek_r1`.
Cela permet de renvoyer, en plus du `content` final, un champ `reasoning_content` qui détaille la chaîne de raisonnement.

Voici un exemple d’appel sur *tous* les endpoints (certains n’auront pas de champ `reasoning_content` si le modèle ne supporte pas le raisonnement).


In [ ]:
def test_reasoning():
    """
    Test du mode reasoning_output (champ 'reasoning_content') 
    s'il est activé sur certains modèles. 
    """
    question = "Combien font 253 * 73 - 287 ?"

    for ep in endpoints:
        logger.info(f"=== Test Reasoning Output sur endpoint='{ep['name']}' ===")

        openai.api_base = ep["api_base"]
        openai.api_key  = ep["api_key"]

        client = OpenAI(
            api_key=ep["api_key"],
            base_url=ep["api_base"]
        )

        try:
            response = client.chat.completions.create(
                model=ep.get("model"),
                messages=[{"role": "user", "content": question}]
            )
        except Exception as ex:
            logger.error(f"  [!] Erreur lors de l'appel endpoint='{ep['name']}': {ex}")
            continue

        choice = response.choices[0]
        msg = choice.message

        reasoning_part = getattr(msg, "reasoning_content", None)
        final_content = msg.content or ""

        logger.debug(f"  -> Réponse (finale) : {final_content[:250]}")
        if reasoning_part:
            logger.info(f"  -> Raisonnement   : {reasoning_part[:250]}")
        else:
            logger.warning("  -> Pas de 'reasoning_content' pour ce modèle/parsing.")

# Lancement
test_reasoning()


### Interprétation du test de parallélisme global

Ce test évalue la **capacité de batching et le débit global** des endpoints :

**Méthodologie** :
1. **25 requêtes par endpoint** lancées simultanément (ordre aléatoire)
2. **Mesure de la fenêtre de concurrence** : du premier démarrage au dernier aboutissement
3. **Calcul du débit effectif** : tokens cumulés / durée totale

**Métriques analysées** :

1. **Débit global** (`global_speed`) :
   - Tous les endpoints combinés
   - Exemple : 500 tok/s si 3 endpoints génèrent en parallèle
   
2. **Débit par endpoint** :
   - Exemple : Endpoint A = 150 tok/s, B = 200 tok/s, C = 150 tok/s
   - Révèle les **différences de performance** sous charge

3. **Fenêtre de concurrency** :
   - Durée entre la première requête lancée et la dernière terminée
   - Plus courte = meilleur throughput

**Observations attendues** :

- **vLLM avec batching** : Débit élevé même avec 25 requêtes simultanées
- **Sans batching** : Dégradation linéaire (25 requêtes = 25× plus lent)
- **OpenAI cloud** : Débit stable mais plafonné par rate limits

**Comparaison séquentiel vs parallèle** :
```
Séquentiel : 30 tok/s × 1 requête = 30 tok/s
Parallèle  : 25 tok/s × 25 requêtes = 625 tok/s total
```

**Conclusion** : Le batching permet de **multiplier le débit global** sans dégrader significativement la latence individuelle. C'est l'avantage majeur de vLLM sur des solutions mono-requête.

## Benchmark final (avec journaux réguliers)

Cette étape exécute un warm-up + N itérations par endpoint.
On calcule ensuite la vitesse tokens/s. 

**Important** : Le prompt est un peu plus long, et la génération peut 
prendre du temps selon la taille du modèle ou la quantization.

Pour ne pas paraître figé, on ajoute des `print` avant et après l'appel, 
pour indiquer la progression.


In [ ]:
import sys
import openai
from openai import OpenAI

# Paramètre: combien de fois on répète la requête pour la mesure.
N_REPEATS = 1
all_results = []

def query_once(api_base, api_key, prompt, model=None):
    """
    Appel unique via la nouvelle API (client = OpenAI(...)) + log.
    Retourne (elapsed_seconds, total_tokens).
    """
    client = OpenAI(api_key=api_key, base_url=api_base)

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    start_time = time.time()
    logger.debug("      ... appel client.chat.completions.create() en cours ...")
    try:
        resp = client.chat.completions.create(
            messages=messages,
            model=model,
            max_completion_tokens=1000,
            stream=False,
        )
        elapsed = time.time() - start_time

        tokens = resp.usage.total_tokens if resp.usage else None
        return elapsed, tokens
    except Exception as e:
        logger.error(f"      [!] Exception: {e}")
        return None, None

def run_benchmark(label, prompt, repeats=N_REPEATS):
    """
    1) Warm-up (unique appel) + N appels chronométrés par endpoint.
    2) On log avant/après chaque appel via logger.
    3) On stocke les métriques dans all_results.
    """
    global all_results

    for ep in endpoints:
        logger.info(f"=== Benchmark: endpoint={ep['name']}, label={label} ===")

        # WARM-UP
        logger.info("   (warm-up) Lancement d'un appel simple.")
        w_elapsed, w_tokens = query_once(
            ep["api_base"], ep["api_key"],
            "Warm up. Ignorez ce message.",
            model=ep.get("model", None)
        )
        if w_elapsed is None:
            logger.warning(f"   [!] Warm-up échoué => skip {ep['name']}.")
            continue

        total_time = 0.0
        total_tokens = 0
        success_count = 0

        logger.info(f"   (benchmark) On va faire {repeats} itérations.")
        for i in range(repeats):
            logger.info(f"   -> Iteration {i+1}/{repeats}")
            e, tks = query_once(
                ep["api_base"], ep["api_key"],
                prompt,
                model=ep.get('model', None)
            )
            if e is None:
                logger.error("      Echec de l'appel, on continue avec le suivant.")
                continue
            logger.info(f"      => Durée: {e:.2f}s, tokens={tks}")
            total_time += e
            if tks is not None:
                total_tokens += tks
            success_count += 1

        if success_count == 0:
            logger.warning("   [!] Aucune itération réussie pour ce endpoint.")
            continue

        avg_time = total_time / success_count
        tok_per_sec = 0.0
        if total_time > 0:
            tok_per_sec = total_tokens / total_time

        res = {
            "label": label,
            "endpoint": ep["name"],
            "repeats": repeats,
            "success_count": success_count,
            "total_time_s": total_time,
            "avg_time_s": avg_time,
            "total_tokens": total_tokens,
            "tokens_per_sec": tok_per_sec
        }
        all_results.append(res)
        logger.info(
            f"   => {ep['name']} OK: {success_count}/{repeats} calls, "
            f"avg_time={avg_time:.2f}s, speed={tok_per_sec:.2f} tok/s"
        )

    logger.info(f"[INFO] Fin du benchmark pour label='{label}'.")

# Exemple d’appel:
label_run = "auto_benchmark1"
USER_PROMPT = (
    "Rédige un texte d'environ 1000 mots sur l'IA, "
    "en évoquant l'apprentissage machine, les grands modèles de langage, "
    "et quelques perspectives d'évolution."
)

run_benchmark(label_run, USER_PROMPT, repeats=1)

logger.info(f"\n=== Résultats de ce premier benchmark (label='{label_run}') ===")
for r in all_results:
    if r["label"] == label_run:
        logger.info(
            f"{r['endpoint']} -> {r['success_count']}/{r['repeats']} ok, "
            f"avg time={r['avg_time_s']:.2f}s, speed={r['tokens_per_sec']:.2f} tok/s"
        )



## Test de traitement parallèle (batching)

Dans cette cellule, nous allons :
- Définir un nombre de requêtes à envoyer en parallèle (`N_PARALLEL`).
- Pour chaque endpoint, lancer ces requêtes en **concurrence**.
- Mesurer le temps total écoulé et le nombre total de tokens.
- Calculer la **vitesse globale** de traitement (tokens / seconde) lorsque plusieurs requêtes arrivent simultanément.

**vLLM** est réputé supporter le batching token-level et donc bénéficier d'une meilleure latence moyenne et d'un meilleur débit lorsqu'il y a plusieurs requêtes en parallèle.


In [ ]:
import aiohttp
import asyncio

N_PARALLEL = 25
PARALLEL_PROMPT = (
    "Bonjour, ceci est un test de requêtes parallèles. "
    "Peux-tu me donner quelques idées créatives pour un week-end ?"
)

async def async_chat_completion(api_base: str, api_key: str, model: str, prompt: str):
    url = f"{api_base}/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 200
    }

    async with aiohttp.ClientSession() as session:
        start_t = time.time()
        try:
            async with session.post(url, headers=headers, json=payload, timeout=60) as resp:
                elapsed = time.time() - start_t
                if resp.status == 200:
                    data = await resp.json()
                    tokens = None
                    if "usage" in data and data["usage"].get("total_tokens"):
                        tokens = data["usage"]["total_tokens"]
                    return (elapsed, tokens)
                else:
                    return (None, None)
        except Exception as e:
            logger.error(f"[!] Exception asynchrone: {e}")
            return (None, None)

async def run_parallel_test(endpoint, n_parallel, prompt):
    api_base = endpoint["api_base"]
    api_key  = endpoint["api_key"]
    model    = endpoint.get("model", None)

    tasks = []
    for _ in range(n_parallel):
        tasks.append(asyncio.create_task(async_chat_completion(api_base, api_key, model, prompt)))

    start_time = time.time()
    results = await asyncio.gather(*tasks)
    total_time = time.time() - start_time

    nb_ok = 0
    sum_tokens = 0
    for (elapsed, tokens) in results:
        if elapsed is not None and tokens is not None:
            nb_ok += 1
            sum_tokens += tokens

    return {
        "endpoint": endpoint["name"],
        "n_req": n_parallel,
        "n_ok": nb_ok,
        "total_time_s": total_time,
        "sum_tokens": sum_tokens
    }

async def parallel_benchmark(endpoints_list, n_parallel, prompt, use_random_prefix=True):
    summary = []
    logger.info(f"==== Lancement du test parallèle: {n_parallel} requêtes simultanées par endpoint ====")
    
    for ep in endpoints_list:
        logger.info(f"=== Parallel test sur '{ep['name']}' ===")
        
        if use_random_prefix:
            prefix = ''.join(random.choices('ABCDEFGHIJKLMNOPQRSTUVWXYZ', k=5))
            modified_prompt = f"{prefix} {prompt}"
        else:
            modified_prompt = prompt
        
        res = await run_parallel_test(ep, n_parallel, modified_prompt)

        nb_ok = res["n_ok"]
        total_time = res["total_time_s"]
        sum_tokens = res["sum_tokens"]
        speed = sum_tokens / total_time if total_time > 0 else 0

        logger.info(f"  -> {nb_ok}/{n_parallel} appels OK")
        logger.info(f"  -> Durée totale (concurrente): {total_time:.2f} s")
        logger.info(f"  -> Tokens cumulés: {sum_tokens}")
        logger.info(f"  -> Vitesse globale: {speed:.2f} tok/s")

        summary.append(res)

    logger.info("\n=== Récapitulatif du test parallèle ===")
    for s in summary:
        speed = 0.0
        if s["total_time_s"] > 0:
            speed = s["sum_tokens"] / s["total_time_s"]
        logger.info(
            f"{s['endpoint']}: {s['n_ok']}/{s['n_req']} ok, "
            f"total_time={s['total_time_s']:.2f}s, speed={speed:.2f} tok/s"
        )

import nest_asyncio
import random
nest_asyncio.apply()

await parallel_benchmark(endpoints, N_PARALLEL, PARALLEL_PROMPT)


## 13 12) Test de parallélisme global : mesures de débits individuels et ordre d’exécution aléatoire

Ici, nous souhaitons :
1. **Mesurer** le débit (tokens/s) **individuellement** pour chaque endpoint.
2. **Lancer** toutes les requêtes (pour tous les endpoints) **en ordre aléatoire**, sans rajouter de délais artificiels.

**Principe** :
- On construit d’abord **la liste** complète des appels (ex. `N_PARALLEL_GLOBAL` requêtes pour chaque endpoint).
- On associe à chaque appel l’endpoint correspondant, puis on **randomise** l’ordre de cette liste.
- On déclenche **en simultané** l’ensemble des requêtes (via `asyncio.gather`).
- Après exécution, on calcule :
  - **Durée totale** (début → fin) pour l’ensemble des requêtes,
  - **Résultats individuels** (tokens cumulés par endpoint, nombre de requêtes OK, etc.),
  - **Débit** de chaque endpoint : (somme des tokens pour cet endpoint) / (durée totale),
  - **Débit global** : (somme de tous les tokens) / (durée totale).


In [ ]:
import aiohttp
import asyncio
import random
import time

N_PARALLEL_GLOBAL = 25  # Nombre de requêtes à envoyer par endpoint
GLOBAL_PROMPT = (
    "Bonjour, ceci est un test de parallélisme global. "
    "Peux-tu me détailler en 500 mots les avantages et inconvénients de travailler "
    "avec plusieurs grands modèles (Llama, Qwen, GPT, etc.) en parallèle sur un même serveur ?"
)

async def async_chat_completion(api_base: str, api_key: str, model: str, prompt: str):
    """
    Effectue une requête unique (POST /chat/completions) en asynchrone.
    Retourne un tuple (start_t, end_t, tokens, success).
      - start_t  : l'instant du début effectif (time.time()) juste avant l'appel
      - end_t    : l'instant de fin (time.time()) juste après la réception de la réponse
      - tokens   : nombre de tokens dans la réponse (None si échec)
      - success  : booléen (True si statut=200 et parse JSON OK)
    """
    # On note l'instant de démarrage avant de créer la session et d'envoyer la requête
    start_t = time.time()

    url = f"{api_base}/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 150
    }

    async with aiohttp.ClientSession() as session:
        try:
            async with session.post(url, headers=headers, json=payload, timeout=60) as resp:
                end_t = time.time()
                if resp.status == 200:
                    data = await resp.json()
                    tokens = data.get("usage", {}).get("total_tokens", None)
                    return (start_t, end_t, tokens, True)
                else:
                    return (start_t, end_t, None, False)
        except Exception as e:
            logger.error(f"[!] Exception asynchrone: {e}")
            end_t = time.time()
            return (start_t, end_t, None, False)

async def run_full_parallel_on_all_endpoints(endpoints, n_parallel, prompt):
    """
    Lance n_parallel requêtes pour chaque endpoint (donc total = n_parallel * len(endpoints)),
    en un seul grand batch concurrent *et dans un ordre aléatoire*.

    Retourne un dict contenant :

    - global_min_start  : le plus petit start_t (sur toutes les requêtes, tous endpoints)
    - global_max_end    : le plus grand end_t   (sur toutes les requêtes)
    - n_req_total       : total de requêtes
    - n_ok_global       : total de requêtes ayant répondu 200 OK
    - sum_tokens_global : total de tokens (toutes requêtes OK)
    - stats_endpoints   : dict par endpoint, contenant :
         {
           "calls": int,
           "ok": int,
           "sum_tokens": int,
           "min_start": float,
           "max_end": float
         }
      (on calcule ensuite un débit = sum_tokens / (max_end - min_start))
    """

    # Prépare tous les appels (endpoint, prompt)
    tasks_info = []
    for ep in endpoints:
        for _ in range(n_parallel):
            # Optionnel : préfixe aléatoire pour "casser" un éventuel cache
            prefix = "".join(random.choices("ABCDEFGHIJKLMNOPQRSTUVWXYZ", k=3))
            modified_prompt = f"{prefix} {prompt}"
            tasks_info.append((ep, modified_prompt))

    # Mélange l’ordre des requêtes
    random.shuffle(tasks_info)

    # Transforme chaque (ep, prompt) en coroutine
    coroutines = []
    for (ep, pr) in tasks_info:
        coroutines.append(async_chat_completion(ep["api_base"], ep["api_key"], ep["model"], pr))

    # Exécution en parallèle
    results = await asyncio.gather(*coroutines)

    # Rassemble toutes les stats
    # On calcule le min de start_t et le max de end_t global
    all_starts = []
    all_ends = []

    stats_per_endpoint = {}
    for ep in endpoints:
        stats_per_endpoint[ep["name"]] = {
            "calls": 0,
            "ok": 0,
            "sum_tokens": 0,
            "min_start": float("inf"),
            "max_end": float("-inf"),
        }

    global_ok = 0
    global_tokens = 0

    for i, (start_t, end_t, tokens, success) in enumerate(results):
        ep_name = tasks_info[i][0]["name"]
        ep_stats = stats_per_endpoint[ep_name]
        ep_stats["calls"] += 1

        # On enrichit le min/max local à l'endpoint
        if start_t < ep_stats["min_start"]:
            ep_stats["min_start"] = start_t
        if end_t > ep_stats["max_end"]:
            ep_stats["max_end"] = end_t

        # Idem pour le global
        all_starts.append(start_t)
        all_ends.append(end_t)

        # On comptabilise tokens si success
        if success and tokens is not None:
            ep_stats["ok"] += 1
            ep_stats["sum_tokens"] += tokens
            global_ok += 1
            global_tokens += tokens

    # min/max global
    global_min_start = min(all_starts) if all_starts else None
    global_max_end = max(all_ends) if all_ends else None

    summary = {
        "global_min_start": global_min_start,
        "global_max_end": global_max_end,
        "n_req_total": len(results),
        "n_ok_global": global_ok,
        "sum_tokens_global": global_tokens,
        "stats_endpoints": stats_per_endpoint
    }
    return summary


# --- Lancement effectif du test ---
logger.info(f"=== Lancement du test de parallélisme global sur {len(endpoints)} endpoints ===")
logger.info(f"   -> {N_PARALLEL_GLOBAL} requêtes par endpoint (ordre aléatoire).")

summary_global = await run_full_parallel_on_all_endpoints(endpoints, N_PARALLEL_GLOBAL, GLOBAL_PROMPT)

# Récapitulatif
global_min_start = summary_global["global_min_start"]
global_max_end = summary_global["global_max_end"]
n_req_total = summary_global["n_req_total"]
n_ok_global = summary_global["n_ok_global"]
sum_tokens_global = summary_global["sum_tokens_global"]

logger.info("")
logger.info("=== Résultats du test global (tous endpoints en même temps) ===")
logger.info(f"  -> Nombre total de requêtes : {n_req_total}")
logger.info(f"  -> Nombre de requêtes OK   : {n_ok_global}")

if global_min_start is not None and global_max_end is not None:
    global_duration = global_max_end - global_min_start
    logger.info(f"  -> Fenêtre de temps (global) : {global_duration:.2f} s")
    logger.info(f"  -> Cumul de tokens (global)  : {sum_tokens_global}")
    if global_duration > 0:
        global_speed = sum_tokens_global / global_duration
        logger.info(f"  -> Débit global (tous endpoints) : {global_speed:.2f} tok/s")

logger.info("\n=== Détails par endpoint ===")
for ep_name, ep_stats in summary_global["stats_endpoints"].items():
    calls = ep_stats["calls"]
    ok = ep_stats["ok"]
    sum_tks = ep_stats["sum_tokens"]
    ep_min_start = ep_stats["min_start"]
    ep_max_end   = ep_stats["max_end"]

    logger.info(f"- {ep_name} : {ok}/{calls} OK, tokens={sum_tks}")
    if ok > 0 and ep_min_start < ep_max_end:  # au moins 1 requête
        ep_concurrency_window = ep_max_end - ep_min_start
        # 'ep_concurrency_window' = fenêtrage du 1er démarrage -> dernier aboutissement
        speed_ep = sum_tks / ep_concurrency_window if ep_concurrency_window > 0 else 0
        logger.info(f"    => Fenêtre concurrency = {ep_concurrency_window:.2f}s")
        logger.info(f"    => Débit effectif ~ {speed_ep:.2f} tok/s")
    else:
        logger.info("    => Pas de requêtes OK ou pas de fenêtre exploitable.")


---

# Comparaison Coût : Cloud vs Local

## Calcul du coût par token

### OpenAI (Cloud)

| Modèle | Input ($/1M tokens) | Output ($/1M tokens) |
|--------|---------------------|----------------------|
| gpt-4o | $2.50 | $10.00 |
| gpt-4o-mini | $0.15 | $0.60 |
| o4-mini | $1.10 | $4.40 |

### Local (estimation)

**Hypothèses** :
- GPU RTX 3090 : ~$1500 amortis sur 3 ans
- Électricité : ~$0.15/kWh, consommation ~350W
- Hébergement : ~200€/mois (serveur dédié) ou gratuit (machine personnelle)

**Calcul pour 1M tokens générés** (DeepSeek R1 8B, ~40 tok/s) :
- Temps : 1M / 40 = 25,000 secondes = ~7 heures
- Électricité : 7h × 0.35kW × $0.15 = **$0.37**
- Amortissement GPU : (7h / 26,280h) × $1500 = **$0.40**
- **Total local : ~$0.77 / 1M tokens**

### Point d'équilibre

```
Volume mensuel où local devient rentable :
- gpt-4o-mini output : $0.60/1M → local rentable dès ~100M tokens/mois
- gpt-4o output : $10.00/1M → local rentable dès ~10M tokens/mois
```

**Conclusion** : Pour un usage intensif (>10M tokens/mois), l'hébergement local est généralement plus économique.

---

# Conclusion

## Points clés

1. **Compatibilité OpenAI API** : vLLM et Ollama exposent la même API que OpenAI
2. **DeepSeek R1** : Alternative locale aux modèles raisonnants (o1, o3)
3. **Qwen 2.5** : Excellent pour le tool calling local
4. **Batching vLLM** : Multiplicateur de débit pour requêtes parallèles
5. **Coût** : Local devient rentable au-delà de ~10M tokens/mois

## Commandes Docker utiles

```bash
# vLLM avec DeepSeek R1
docker run --gpus all -p 8000:8000 \
  vllm/vllm-openai:latest \
  --model deepseek-ai/DeepSeek-R1-Distill-Llama-8B \
  --enable-reasoning --reasoning-parser deepseek_r1

# Ollama (plus simple)
docker run -d --gpus all -p 11434:11434 ollama/ollama
ollama pull deepseek-r1:8b
```

## Ressources

- [vLLM Documentation](https://docs.vllm.ai/)
- [Ollama](https://ollama.ai/)
- [DeepSeek R1](https://github.com/deepseek-ai/DeepSeek-R1)
- [Qwen 2.5](https://github.com/QwenLM/Qwen2.5)

## Prochaines étapes

- **Notebook 8** : Comparaison détaillée des modèles raisonnants
- **Notebook 9** : Patterns de production (retry, monitoring)